# Dipole Antenna Simulation with openEMS

This notebook demonstrates a half-wave dipole simulation using openEMS.
Features included:
- S11 sweep across a frequency band
- Far-field radiation pattern (polar and 3D)
- Optional ground plane for realism
- Export of field data to ParaView

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from openems import openEMS
from openems.geometry import CSX, RectWire, Box
from openems.excitation import GaussianExcitation
from openems.postprocess import CalcS11, PlotS11, CalcFarField, PlotFarField

# --- Simulation Parameters ---
f0 = 3e9                      # Center frequency: 3 GHz
c = 3e8                       # Speed of light
lambda0 = c / f0              # Wavelength
dipole_length = lambda0 / 2   # Half-wave dipole
dipole_radius = 0.001         # Dipole radius
unit = 1e-3                   # Unit in meters
sim_path = "./dipole_sim"
if not os.path.exists(sim_path):
    os.makedirs(sim_path)

ModuleNotFoundError: No module named 'openems'

In [ ]:
csx = CSX()

# Add dipole arms
csx.Add(RectWire('copper', dipole_radius, [0, 0, 0], [0, 0, dipole_length/2]))
csx.Add(RectWire('copper', dipole_radius, [0, 0, 0], [0, 0, -dipole_length/2]))

# Optional: Add ground plane (uncomment to enable realism)
csx.Add(Box('PEC', start=[-0.1, -0.1, -lambda0], stop=[0.1, 0.1, -lambda0 + 0.001]))

# Excitation setup
GaussianExcitation(csx, port_number=1, start=[0, 0, -dipole_length/2], stop=[0, 0, dipole_length/2], f0=f0)

In [ ]:
# --- Simulation Setup ---
sim = openEMS()
sim.SetFDTD(time_step=1e-12, end_time=100e-9)
sim.SetCSX(csx)
sim.SetMesh([[-100, 100], [-100, 100], [-100, 100]], unit=unit)
sim.AddExcitation(1)
sim.Run(path=sim_path, name="dipole")

In [ ]:
# --- S11 Frequency Sweep ---
s11 = CalcS11(f"{sim_path}/dipole")
frequencies = s11[:, 0] / 1e9
s11_vals = 20 * np.log10(np.abs(s11[:, 1]))

plt.figure()
plt.plot(frequencies, s11_vals)
plt.title("S11 (Return Loss)")
plt.xlabel("Frequency (GHz)")
plt.ylabel("S11 (dB)")
plt.grid(True)
plt.show()

In [ ]:
# --- Far-field Calculation ---
theta_range = np.linspace(0, 180, 361)
phi = 0
far_field = CalcFarField(f"{sim_path}/dipole", f0, theta_range, phi)

# Plot 2D Far-field
PlotFarField(far_field, theta_range, quantity="Etheta", unit="dB", title="2D Far-field Pattern")

In [ ]:
# --- 3D Radiation Pattern ---
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
theta_rad = np.radians(theta_range)
r = np.abs(far_field["Etheta"])

x = r * np.sin(theta_rad) * np.cos(phi)
y = r * np.sin(theta_rad) * np.sin(phi)
z = r * np.cos(theta_rad)

ax.plot_surface(x.reshape(1, -1), y.reshape(1, -1), z.reshape(1, -1), color='blue', alpha=0.6)
ax.set_title("3D Dipole Radiation Pattern")
plt.show()

In [ ]:
# --- Export Fields for ParaView (VTK Format) ---
sim.ExportVTK(f"{sim_path}/fields")
print("Field data exported to VTK format for ParaView visualization.")